# Documentation of Study Plans

## Image viewing parameters/environment
1. Custom viewer from gaze point
2. No windowing
3. No dicoms (images will be in .png)
4. Full screen images (max 24inch screen)
5. No portrait
6. Routine radiology read with free dictation (no typing)
7. Key anatomies and devices need to be reported about

## Inclusion criteria considerations
1. Age range: ? depends on final target population - should do some exploratory analysis of metadata first
2. Gender: raw ED population distribution or half-half? Balanced dataset vs original population distribution? Given limited number of images in the pilot dataset, balanced dataset is more likely to capture a wider feature variation within different strata of target population.
3. Admission severity: same as for age - may be an useful indicator of CHF severity if we want to grade CHF
4. Technically not too hard...and...not too easy cases.
5. PA only - AP may introduce more heterogeneity amongst annotators due to poorer quality of images. PA only is a good idea for a pilot after consulting radiologists.

## Exclusion criteria considerations
1. Patient admitted to ICU (too sick possibly too difficult, bad quality imaging (portables))
2. No laterals for this pilot project since can only view 1 image at a time

## Diagnoses selected for eye gaze dataset after above considerations
1. CHF (congestive heart failure) - grading severity, both Arjun and Ali think this will be really interesting to focus on
2. Pneumonia - over annotated by existing efforts already
3. Normals - must have

## Other interesting diseases that were initially considered but got dropped
1. Pneumothorax - requires windowing capability (not allowed by eye gaze software) to report accurately
2. Nodules - not enough cases in ED population
3. Tubes and lines placement (endotracheal tube, central vascular lines, enteric tubes) - very difficult problem technically

## Notes from MIMIC contact
1. All docs are online now: https://mimic-iv.mit.edu/ - getting comprehensive, but please raise any issues you find here http://github.com/mit-lcp/mimic-iv-website
2. Recommend using subject_id - everybody will have the same subject_id across datasets. If you want to match a stay_id, then use subject_id and the intime/outtime in your join from the stays table.


# Setups

In [ ]:
# Required files
import os
import sys
import pandas as pd
import pandas_gbq
import re

# Connect colab with google bigquery stuff
from google.colab import auth
from google.cloud import bigquery
from google.colab import files
from google.colab import drive

### Required MIMIC derived tables

You will need to get access to the following Eye Gaze project related derived tables to run this notebook.
1. eye_gaze.report_normal_cxr
2. eye_gaze.icd_criteria


# Authentications

In [ ]:
auth.authenticate_user()

In [ ]:
# Specify your project_id on Google Cloud
project_id='mimic-projects'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

# Selecting patients with ED meta data

In [ ]:
# Summarises mimic_ed patient and stay info
sql = """
SELECT 
count(distinct subject_id) as patients
,count(distinct stay_id) as ed_stays
FROM `physionet-data.mimic_ed.diagnosis`
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id, dialect = 'standard')
df

,patients,ed_stays
0,200529,408856


In [ ]:
# Summarises unique ED patients in MIMIC CXR dataset
sql = """
SELECT  
count(distinct subject_id) as patients
, count(distinct study_id) as exams
, count(distinct dicom_id) as images
FROM `physionet-data.mimic_cxr.record_list`
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id, dialect = 'standard')
df

,patients,exams,images
0,65379,227835,377110


In [ ]:
# Linking MIMIC CXR with MIMIC ED data, along with ICD-9 diagnoses
# This query no longer runs as one of the MIMIC source tables has been updated.
sql = """
WITH

-- Get the useful dicom metadata for image level data selection
dicom_meta AS (
SELECT dicom AS dicom_id, rec.path
    , CAST(StudyID AS int64) AS study_id
    , CAST(PatientID AS int64) AS patient_id
    , StudyDate
    , StudyTime
    --, CONCAT(StudyDate,SPLIT(StudyTime, '.')[OFFSET(0)]) AS StudyDateTime
    , PARSE_TIMESTAMP('%Y%m%d%H%M%S', CONCAT(StudyDate,SPLIT(StudyTime, '.')[OFFSET(0)])) AS StudyDateTime
    , EXTRACT(DATE FROM PARSE_TIMESTAMP('%Y%m%d%H%M%S', CONCAT(StudyDate,SPLIT(StudyTime, '.')[OFFSET(0)]))) AS cxrdate
    , ViewPosition
    , RequestedProcedureDescription
    , RequestedProcedurePriority
    , ExposedArea, PatientOrientation
    , Modality, BodyPartExamined, WindowCenter, WindowWidth, RescaleIntercept, Rescaleslope, RescaleType, BurnedInAnnotation
FROM `physionet-data.mimic_cxr.dicom_metadata_string` AS di
INNER JOIN `physionet-data.mimic_cxr.record_list` AS rec
    ON di.dicom = rec.dicom_id
WHERE BodyPartExamined LIKE '%CHEST%'
    AND BurnedInAnnotation LIKE 'NO'
)

-- Get ED admit info
, ed_admit AS (
SELECT
    * --, could add the dx codes here in future
    , DENSE_RANK() OVER (PARTITION BY subject_id ORDER BY intime) AS edstay_seq
    , EXTRACT(DATE FROM intime) AS indate
    , EXTRACT(DATE FROM outtime) AS outdate
FROM `physionet-data.mimic_ed.main`
--FROM `physionet-data.mimic_core.transfers`
)

-- Match CXR study_id with ED stay_id
, cxr2ed_admit AS (
SELECT cxr.*
    , ed.* --, could add the dx codes here in future
FROM dicom_meta AS cxr
INNER JOIN ed_admit AS ed
    ON ed.subject_id = cxr.patient_id
    --AND cxr.StudyDateTime BETWEEN ed.intime AND ed.outtime
    AND TIMESTAMP(cxr.cxrdate) BETWEEN TIMESTAMP(ed.indate) AND TIMESTAMP(ed.outdate)
)

-- Construct final table
, final AS (
SELECT cxr.*
    , adm.gender, adm.anchor_age, adm.anchor_year, adm.anchor_year_shifted, adm.dod
FROM cxr2ed_admit AS cxr
-- include some admission level information
INNER JOIN `physionet-data.mimic_core.patients` AS adm
    ON cxr.patient_id = adm.subject_id
)

SELECT * FROM final
--WHERE ViewPosition LIKE "AP" OR ViewPosition LIKE "PA"

--SELECT StudyDateTime FROM dicom_meta
--SELECT PARSE_TIMESTAMP('%Y%m%d%H%M%S', StudyDateTime) FROM dicom_meta
--SELECT CAST(StudyDate AS TIME) FROM dicom_meta
--SELECT intime FROM `physionet-data.mimic_ed.main`
--WHERE dicom_id IN (SELECT dicom_id FROM `physionet-data.mimic_cxr.record_list`)

--SELECT * FROM dicom_meta
--SELECT * FROM cxr2ed_admit

"""



In [ ]:
# This query is updated to adapt to changes with the new MIMIC ed tables in BigQuery
sql = """
WITH

dx_keys AS(
SELECT 
  subject_id, stay_id, icd_title, icd_code
  , CONCAT('dx',seq_num) as key1
  , CONCAT('dx',seq_num,'_icd') as key2
FROM `physionet-data.mimic_ed.diagnosis`
)

, diagnosis AS(
SELECT 
  subject_id, stay_id, 
  MAX(IF(key1 = "dx1", icd_title, NULL)) AS dx1,
  MAX(IF(key2 = "dx1_icd", icd_code, NULL)) AS dx1_icd,
  MAX(IF(key1 = "dx2", icd_title, NULL)) AS dx2,
  MAX(IF(key2 = "dx2_icd", icd_code, NULL)) AS dx2_icd,
  MAX(IF(key1 = "dx3", icd_title, NULL)) AS dx3,
  MAX(IF(key2 = "dx3_icd", icd_code, NULL)) AS dx3_icd,
  MAX(IF(key1 = "dx4", icd_title, NULL)) AS dx4,
  MAX(IF(key2 = "dx4_icd", icd_code, NULL)) AS dx4_icd,
  MAX(IF(key1 = "dx5", icd_title, NULL)) AS dx5,
  MAX(IF(key2 = "dx5_icd", icd_code, NULL)) AS dx5_icd,
  MAX(IF(key1 = "dx6", icd_title, NULL)) AS dx6,
  MAX(IF(key2 = "dx6_icd", icd_code, NULL)) AS dx6_icd,
  MAX(IF(key1 = "dx7", icd_title, NULL)) AS dx7,
  MAX(IF(key2 = "dx7_icd", icd_code, NULL)) AS dx7_icd,
  MAX(IF(key1 = "dx8", icd_title, NULL)) AS dx8,
  MAX(IF(key2 = "dx8_icd", icd_code, NULL)) AS dx8_icd,
  MAX(IF(key1 = "dx9", icd_title, NULL)) AS dx9,
  MAX(IF(key2 = "dx9_icd", icd_code, NULL)) AS dx9_icd,
FROM dx_keys 
GROUP BY subject_id, stay_id 
ORDER BY subject_id, stay_id
)
-- SELECT
-- count(distinct subject_id) --200529
-- , count(distinct stay_id) -- 408856
-- FROM diagnosis
--SELECT * FROM diagnosis

-- Get ED admit info
, ed_admit AS (
SELECT
    ed.* --, could add the dx codes here in future
    , DENSE_RANK() OVER (PARTITION BY ed.subject_id ORDER BY ed.intime) AS edstay_seq
    , EXTRACT(DATE FROM ed.intime) AS indate
    , EXTRACT(DATE FROM ed.outtime) AS outdate
    , dx.dx1, dx.dx1_icd
    , dx.dx2, dx.dx2_icd
    , dx.dx3, dx.dx3_icd
    , dx.dx4, dx.dx4_icd
    , dx.dx5, dx.dx5_icd
    , dx.dx6, dx.dx6_icd
    , dx.dx7, dx.dx7_icd
    , dx.dx8, dx.dx8_icd
    , dx.dx9, dx.dx9_icd
FROM `physionet-data.mimic_ed.edstays` as ed
LEFT JOIN diagnosis as dx
ON ed.stay_id = dx.stay_id
--FROM `physionet-data.mimic_ed.main`
--FROM `physionet-data.mimic_core.transfers`
)
--SELECT * FROM ed_admit

-- Get the useful dicom metadata for image level data selection
, dicom_meta AS (
SELECT dicom AS dicom_id, rec.path
    , CAST(StudyID AS int64) AS study_id
    , CAST(PatientID AS int64) AS patient_id
    , StudyDate
    , StudyTime
    --, CONCAT(StudyDate,SPLIT(StudyTime, '.')[OFFSET(0)]) AS StudyDateTime
    , PARSE_TIMESTAMP('%Y%m%d%H%M%S', CONCAT(StudyDate,SPLIT(StudyTime, '.')[OFFSET(0)])) AS StudyDateTime
    , EXTRACT(DATE FROM PARSE_TIMESTAMP('%Y%m%d%H%M%S', CONCAT(StudyDate,SPLIT(StudyTime, '.')[OFFSET(0)]))) AS cxrdate
    , ViewPosition
    , RequestedProcedureDescription
    , RequestedProcedurePriority
    , ExposedArea, PatientOrientation
    , Modality, BodyPartExamined, WindowCenter, WindowWidth, RescaleIntercept, Rescaleslope, RescaleType, BurnedInAnnotation
FROM `physionet-data.mimic_cxr.dicom_metadata_string` AS di
INNER JOIN `physionet-data.mimic_cxr.record_list` AS rec
    ON di.dicom = rec.dicom_id
WHERE BodyPartExamined LIKE '%CHEST%'
    AND BurnedInAnnotation LIKE 'NO'
)

-- Match CXR study_id with ED stay_id
, cxr2ed_admit AS (
SELECT cxr.*
    , ed.* --, could add the dx codes here in future
FROM dicom_meta AS cxr
INNER JOIN ed_admit AS ed
    ON ed.subject_id = cxr.patient_id
    --AND cxr.StudyDateTime BETWEEN ed.intime AND ed.outtime
    AND TIMESTAMP(cxr.cxrdate) BETWEEN TIMESTAMP(ed.indate) AND TIMESTAMP(ed.outdate)
)

-- Construct final table
, final AS (
SELECT cxr.*
    , adm.gender, adm.anchor_age, adm.anchor_year, adm.anchor_year_group, adm.dod
FROM cxr2ed_admit AS cxr
-- include some admission level information
INNER JOIN `physionet-data.mimic_core.patients` AS adm
    ON cxr.patient_id = adm.subject_id
)

SELECT * FROM final
"""

data = pandas_gbq.read_gbq(sql, project_id=project_id, dialect = 'standard')
print(data.shape)

(150503, 52)


In [ ]:
data.head()

,dicom_id,path,study_id,patient_id,StudyDate,StudyTime,StudyDateTime,cxrdate,ViewPosition,RequestedProcedureDescription,RequestedProcedurePriority,ExposedArea,PatientOrientation,Modality,BodyPartExamined,WindowCenter,WindowWidth,RescaleIntercept,Rescaleslope,RescaleType,BurnedInAnnotation,subject_id,hadm_id,stay_id,intime,outtime,edstay_seq,indate,outdate,dx1,dx1_icd,dx2,dx2_icd,dx3,dx3_icd,dx4,dx4_icd,dx5,dx5_icd,dx6,dx6_icd,dx7,dx7_icd,dx8,dx8_icd,dx9,dx9_icd,gender,anchor_age,anchor_year,anchor_year_group,dod
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,files/p10/p10000032/s50414267/02aa804e-bde0afd...,50414267,10000032,21800506,213014.531,2180-05-06 21:30:14+00:00,2180-05-06,PA,None,None,"[35, 32]","['R', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000032,22595853.0,30486003,2180-05-06 19:17:00,2180-05-06 23:30:00,1,2180-05-06,2180-05-06,"OTH SEQUELA, CHR LIV DIS",572.8,OTHER ASCITES,789.59,UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...,070.70,ASYMPTOMATIC HIV INFECTION,V08,None,None,None,None,None,None,None,None,None,None,F,52,2180,2014 - 2016,2180-09-09
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,files/p10/p10000032/s50414267/174413ec-4ec4c1f...,50414267,10000032,21800506,213014.531,2180-05-06 21:30:14+00:00,2180-05-06,LATERAL,None,None,"[37, 30]","['A', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000032,22595853.0,30486003,2180-05-06 19:17:00,2180-05-06 23:30:00,1,2180-05-06,2180-05-06,"OTH SEQUELA, CHR LIV DIS",572.8,OTHER ASCITES,789.59,UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...,070.70,ASYMPTOMATIC HIV INFECTION,V08,None,None,None,None,None,None,None,None,None,None,F,52,2180,2014 - 2016,2180-09-09
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,53189527,10000032,21800626,165500.312,2180-06-26 16:55:00+00:00,2180-06-26,PA,None,None,"[42, 35]","['R', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000032,22841357.0,38924341,2180-06-26 15:54:00,2180-06-26 21:31:00,2,2180-06-26,2180-06-26,OTHER ASCITES,789.59,UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...,070.70,CIRRHOSIS OF LIVER NOS,571.5,ASYMPTOMATIC HIV INFECTION,V08,None,None,None,None,None,None,None,None,None,None,F,52,2180,2014 - 2016,2180-09-09
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,files/p10/p10000032/s53189527/e084de3b-be89b11...,53189527,10000032,21800626,165500.312,2180-06-26 16:55:00+00:00,2180-06-26,LATERAL,None,None,"[42, 34]","['A', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000032,22841357.0,38924341,2180-06-26 15:54:00,2180-06-26 21:31:00,2,2180-06-26,2180-06-26,OTHER ASCITES,789.59,UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...,070.70,CIRRHOSIS OF LIVER NOS,571.5,ASYMPTOMATIC HIV INFECTION,V08,None,None,None,None,None,None,None,None,None,None,F,52,2180,2014 - 2016,2180-09-09
4,3be619d1-506a66cf-ff1ab8a1-2efb77bb-fe7d59fc,files/p10/p10000935/s51178377/3be619d1-506a66c...,51178377,10000935,21870823,191426.062,2187-08-23 19:14:26+00:00,2187-08-23,LATERAL,None,None,"[38, 28]","['A', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000935,26381316.0,38097512,2187-08-23 14:37:00,2187-08-23 22:46:00,3,2187-08-23,2187-08-23,VOMITING,787.03,OTHER MALAISE AND FATIGUE,780.79,EDEMA,782.3,None,None,None,None,None,None,None,None,None,None,None,None,F,52,2182,2008 - 2010,NaT


# Derive normal class from CXR reports and ICD diagnosis code

In [ ]:
# Get a list of "normal" CXR according to prior NLP analysis of the associated CXR reports

sqlnorm = """
SELECT * FROM `"""+project_id+""".eye_gaze.report_normal_cxr`
"""

normal = pandas_gbq.read_gbq(sqlnorm, project_id=project_id, dialect = 'standard')
print(normal.shape)

(53153, 6)


In [ ]:
normal.head()

,Source,subject_id,rad_id,view,position,consensus_Labels
0,MIMIC,66560,59688011,Frontal,AP,normal anatomically
1,MIMIC,70454016,53869017,Frontal,AP,normal anatomically
2,MIMIC,73600512,59460842,Frontal,AP,normal anatomically
3,MIMIC,73600512,58300375,Frontal,AP,normal anatomically
4,MIMIC,79110144,53445454,Frontal,AP,normal anatomically


In [ ]:
overlap = set(data.study_id).intersection(set(normal.rad_id))
print(len(overlap))

24906


In [ ]:
normIDs = list(set(normal.rad_id))
print(len(normIDs))

print(data.shape)
data['normal_reports'] = [1 if x in normIDs else 0 for x in data.study_id]
print(data.shape)
data.head(5)

48714
(147808, 52)
(147808, 53)


,dicom_id,path,study_id,patient_id,StudyDate,StudyTime,StudyDateTime,cxrdate,ViewPosition,RequestedProcedureDescription,RequestedProcedurePriority,ExposedArea,PatientOrientation,Modality,BodyPartExamined,WindowCenter,WindowWidth,RescaleIntercept,Rescaleslope,RescaleType,BurnedInAnnotation,subject_id,stay_id,intime,outtime,sex,dx1,dx1_icd,dx2,dx2_icd,dx3,dx3_icd,dx4,dx4_icd,dx5,dx5_icd,dx6,dx6_icd,dx7,dx7_icd,dx8,dx8_icd,dx9,dx9_icd,edstay_seq,indate,outdate,gender,anchor_age,anchor_year,anchor_year_shifted,dod,normal_reports
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,files/p10/p10000032/s50414267/02aa804e-bde0afd...,50414267,10000032,21800506,213014.531,2180-05-06 21:30:14+00:00,2180-05-06,PA,None,None,"[35, 32]","['R', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000032,31333364,2180-05-06 19:22:00+00:00,2180-05-06 22:24:00+00:00,F,"OTH SEQUELA, CHR LIV DIS",572.8,OTHER ASCITES,789.59,UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...,070.70,ASYMPTOMATIC HIV INFECTION,V08,None,None,None,None,None,None,None,None,None,None,1,2180-05-06,2180-05-06,F,50 - 60,2013,2180,2180-09-09,0
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,files/p10/p10000032/s50414267/174413ec-4ec4c1f...,50414267,10000032,21800506,213014.531,2180-05-06 21:30:14+00:00,2180-05-06,LATERAL,None,None,"[37, 30]","['A', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000032,31333364,2180-05-06 19:22:00+00:00,2180-05-06 22:24:00+00:00,F,"OTH SEQUELA, CHR LIV DIS",572.8,OTHER ASCITES,789.59,UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...,070.70,ASYMPTOMATIC HIV INFECTION,V08,None,None,None,None,None,None,None,None,None,None,1,2180-05-06,2180-05-06,F,50 - 60,2013,2180,2180-09-09,0
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,53189527,10000032,21800626,165500.312,2180-06-26 16:55:00+00:00,2180-06-26,PA,None,None,"[42, 35]","['R', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000032,36023266,2180-06-26 16:22:00+00:00,2180-06-26 18:28:00+00:00,F,OTHER ASCITES,789.59,UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...,070.70,CIRRHOSIS OF LIVER NOS,571.5,ASYMPTOMATIC HIV INFECTION,V08,None,None,None,None,None,None,None,None,None,None,2,2180-06-26,2180-06-26,F,50 - 60,2013,2180,2180-09-09,0
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,files/p10/p10000032/s53189527/e084de3b-be89b11...,53189527,10000032,21800626,165500.312,2180-06-26 16:55:00+00:00,2180-06-26,LATERAL,None,None,"[42, 34]","['A', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000032,36023266,2180-06-26 16:22:00+00:00,2180-06-26 18:28:00+00:00,F,OTHER ASCITES,789.59,UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...,070.70,CIRRHOSIS OF LIVER NOS,571.5,ASYMPTOMATIC HIV INFECTION,V08,None,None,None,None,None,None,None,None,None,None,2,2180-06-26,2180-06-26,F,50 - 60,2013,2180,2180-09-09,0
4,3be619d1-506a66cf-ff1ab8a1-2efb77bb-fe7d59fc,files/p10/p10000935/s51178377/3be619d1-506a66c...,51178377,10000935,21870823,191426.062,2187-08-23 19:14:26+00:00,2187-08-23,LATERAL,None,None,"[38, 28]","['A', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000935,31989916,2187-08-23 16:50:00+00:00,2187-08-23 21:23:00+00:00,F,VOMITING,787.03,OTHER MALAISE AND FATIGUE,780.79,EDEMA,782.3,None,None,None,None,None,None,None,None,None,None,None,None,3,2187-08-23,2187-08-23,F,40 - 50,1997,2171,NaT,0


In [ ]:
# Exclude cases where the ICD code associated with the ED admission indicate a disease diagnosis that would most likely make the patient's CXR abnormal
# The list of exclusion ICD were manually picked by a clinician (Joy)
sqlexclude = """
SELECT * FROM `"""+project_id+""".eye_gaze.icd_criteria`
"""

exclude = pandas_gbq.read_gbq(sqlexclude, project_id=project_id, dialect = 'standard')

print(exclude.shape)

exclude.head()

(4431, 4)


,icd,description,exclude_normal_cxr,possible_association_with_abnormal_findings
0,88.82,babesiosis,0,1
1,38.9,septicemia nos,0,1
2,41.12,methicillin resistant staphylococcus aureus in...,0,1
3,41.85,other gram negative infection,0,1
4,41.89,other spec bacteria infection,0,1


In [ ]:
excludeICD = list(set(exclude[exclude['exclude_normal_cxr']==1].icd))
print(len(excludeICD))
print(excludeICD[:10])

exclude[exclude['exclude_normal_cxr']==1].head()

357
['S42.031A', '807.07', 'S32.049A', '514', 'S42.152A', '197.1', 'S22.32XA', '518.81', 'I50.43', 'S22.41XA']


,icd,description,exclude_normal_cxr,possible_association_with_abnormal_findings
4074,11.94,pulmon tb nos-cult dx,1,0
4075,31.2,dmac,1,0
4076,41.81,mycoplasma,1,0
4077,112.81,candidal endocarditis,1,0
4078,112.84,candidal esophagitis,1,0


In [ ]:
# Add a column to indicate which row/stay_id should be excluded from the final "Normal" class
print(data.shape)
data['exclude_normal_cxr'] = 0*data.shape[1]
for n in range(1,10):
    data['exclude_normal_cxr'] = [1 if str(x) in excludeICD else y for (x,y) in zip(data['dx'+str(n)+'_icd'],data['exclude_normal_cxr'])]
    print(n)
print(data.shape)

(147808, 54)
1
2
3
4
5
6
7
8
9
(147808, 54)


In [ ]:
# Add final normal class column
print(data.shape)
data['Normal'] = [1 if (x==1)&(y==0) else 0 for (x,y) in zip(data['normal_reports'],data['exclude_normal_cxr'])]
print(data.shape)

(147808, 54)
(147808, 55)


In [ ]:
normals = data[data['Normal'] == 1].reset_index(drop=True).copy()
print(normals.shape)
normals.head()

(49262, 55)


,dicom_id,path,study_id,patient_id,StudyDate,StudyTime,StudyDateTime,cxrdate,ViewPosition,RequestedProcedureDescription,RequestedProcedurePriority,ExposedArea,PatientOrientation,Modality,BodyPartExamined,WindowCenter,WindowWidth,RescaleIntercept,Rescaleslope,RescaleType,BurnedInAnnotation,subject_id,stay_id,intime,outtime,sex,dx1,dx1_icd,dx2,dx2_icd,dx3,dx3_icd,dx4,dx4_icd,dx5,dx5_icd,dx6,dx6_icd,dx7,dx7_icd,dx8,dx8_icd,dx9,dx9_icd,edstay_seq,indate,outdate,gender,anchor_age,anchor_year,anchor_year_shifted,dod,normal_reports,exclude_normal_cxr,Normal
0,28fad2ac-d6001216-b4f72c5b-2d4d452e-17b6c9a5,files/p10/p10001038/s58224503/28fad2ac-d600121...,58224503,10001038,21490807,090708.109,2149-08-07 09:07:08+00:00,2149-08-07,PA,None,None,"[42, 35]","['R', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10001038,36767874,2149-08-07 08:51:00+00:00,2149-08-07 08:55:00+00:00,M,UNSPEC VIRAL INFECTION,079.99,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,2149-08-07,2149-08-07,M,20 - 30,2015,2149,NaT,1,0,1
1,5cae71aa-99bb4662-41ef629e-7e89308c-5831ff9c,files/p10/p10001038/s58224503/5cae71aa-99bb466...,58224503,10001038,21490807,090708.109,2149-08-07 09:07:08+00:00,2149-08-07,PA,None,None,"[42, 35]","['R', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10001038,36767874,2149-08-07 08:51:00+00:00,2149-08-07 08:55:00+00:00,M,UNSPEC VIRAL INFECTION,079.99,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,2149-08-07,2149-08-07,M,20 - 30,2015,2149,NaT,1,0,1
2,f3e86afd-437eb86e-0f9bafb8-1e0f6f96-084ac17e,files/p10/p10001038/s58224503/f3e86afd-437eb86...,58224503,10001038,21490807,090708.109,2149-08-07 09:07:08+00:00,2149-08-07,LATERAL,None,None,"[42, 35]","['A', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10001038,36767874,2149-08-07 08:51:00+00:00,2149-08-07 08:55:00+00:00,M,UNSPEC VIRAL INFECTION,079.99,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,2149-08-07,2149-08-07,M,20 - 30,2015,2149,NaT,1,0,1
3,7f885b34-699f595b-36446207-4f146366-55dfb63f,files/p10/p10001884/s57166219/7f885b34-699f595...,57166219,10001884,21301005,131754.953,2130-10-05 13:17:54+00:00,2130-10-05,PA,None,None,"[42, 35]","['R', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10001884,36566328,2130-10-05 15:34:00+00:00,2130-10-05 20:04:00+00:00,F,"Dyspnea, unspecified",R06.00,Cough,R05,Hypokalemia,E87.6,None,None,None,None,None,None,None,None,None,None,None,None,9,2130-10-05,2130-10-05,F,50 - 60,2000,2115,NaT,1,0,1
4,9cd94646-ca29ae78-e02e6b81-eca7a7f0-1d039a3e,files/p10/p10001884/s57166219/9cd94646-ca29ae7...,57166219,10001884,21301005,131754.953,2130-10-05 13:17:54+00:00,2130-10-05,LATERAL,None,None,"[41, 35]","['A', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10001884,36566328,2130-10-05 15:34:00+00:00,2130-10-05 20:04:00+00:00,F,"Dyspnea, unspecified",R06.00,Cough,R05,Hypokalemia,E87.6,None,None,None,None,None,None,None,None,None,None,None,None,9,2130-10-05,2130-10-05,F,50 - 60,2000,2115,NaT,1,0,1


In [ ]:
print(len(data[data['normal_reports'] == 1]))
print(len(data[data['exclude_normal_cxr'] == 1]))
print(len(data[data['Normal'] == 1]))

52893
23128
49262


# Add columns for other target disease classes

In [ ]:
# Print unique top 3 diagnoses
for dx in list(set(data['dx1']).union(set(data['dx2'])).union(set(data['dx3']))):
    print(dx)

PROTEINURIA
ADV EFF URIC ACID METAB
Laceration w/o foreign body of oth part of head, init encntr
Alcoholic cirrhosis of liver with ascites
ACC POISN-ISOPROPYL ALC
Age-related cognitive decline
Contact w and exposure to infect w a sexl mode of transmiss
OTHER LATE EFFECT CVA
Gastro-esophageal laceration-hemorrhage syndrome
PHLEBITIS & THROMBOPHLEBITIS OF INTRACRANIAL SINUS
Contusion of left wrist, initial encounter
Bariatric surgery status
Other complications of gastric band procedure
Poisoning by oth narcotics, accidental (unintentional), init
Unsp opn wnd l frnt wl of thorax w/o penet thor cavity, init
Hemothorax
HYPER HRT DIS, UNSPEC WITH HEART FAILURE
CONTUSION UPPER LIMB NOS
FX ANKLE NOS-CLOSED
Cannabis use, unspecified, uncomplicated
Pathological fracture, other site, init encntr for fracture
TOXIC ENCEPHALOPATHY
Other speech disturbances
Cardiomyopathy, unspecified
ACUTE ANGL-CLOS GLAUCOMA
BARBITURATE ABUSE-UNSPEC
8 weeks gestation of pregnancy
POSTSURGICAL STATES NEC
Disp fx of 

In [ ]:
# Spot with word boundary detection
def reSpotter(text, lexicons):
    for term in lexicons:
        mre = r'\b' + re.escape(term) + r'\b'
        if (re.search(mre, text, flags=re.IGNORECASE) != None):
            #spot = text.find(term)
            return term
    return False

diagnosis = { #match phrases all in lower cases
             
         'CHF':['heart failure','chf','congestive failure','cardiac failure']
         , 'pneumonia':['pneumonia','chest infection','lung infection']
         , 'copd' : ['copd','chronic obstructive']
         , 'sob' : ['shortness of breath','dyspnea','sob','short of breath']
       }

for i in data.index:
    dxlist = []
    for j in range(1,9):
        col = 'dx' + str(j)
        dx = data.loc[i,col]
        if (dx != None)&(str(dx) != ''):
            dxlist.append(dx.lower()) #lower case before matching
    for key, phrases in diagnosis.items():
        present = 0
        for dx in dxlist:
            if reSpotter(' '+dx+' ', phrases) != False:
                # Only include none negated cases
                if reSpotter(' '+dx+' ', ['without','no','negative']) == False:
                    present = 1
        if present == 1:
            data.loc[i,key] = 1
        else:
            data.loc[i,key] = 0
    if i % 1000 ==0:
        print(i)

print('Done')


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
Done


In [ ]:
print(data.shape)
data['CHF'] = [int(x) for x in data['CHF']]
data['pneumonia'] = [int(x) for x in data['pneumonia']]
data['copd'] = [int(x) for x in data['copd']]
data['sob'] = [int(x) for x in data['sob']]
data.head(5)

(147808, 59)


,dicom_id,path,study_id,patient_id,StudyDate,StudyTime,StudyDateTime,cxrdate,ViewPosition,RequestedProcedureDescription,RequestedProcedurePriority,ExposedArea,PatientOrientation,Modality,BodyPartExamined,WindowCenter,WindowWidth,RescaleIntercept,Rescaleslope,RescaleType,BurnedInAnnotation,subject_id,stay_id,intime,outtime,sex,dx1,dx1_icd,dx2,dx2_icd,dx3,dx3_icd,dx4,dx4_icd,dx5,dx5_icd,dx6,dx6_icd,dx7,dx7_icd,dx8,dx8_icd,dx9,dx9_icd,edstay_seq,indate,outdate,gender,anchor_age,anchor_year,anchor_year_shifted,dod,normal_reports,exclude_normal_cxr,Normal,CHF,pneumonia,copd,sob
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,files/p10/p10000032/s50414267/02aa804e-bde0afd...,50414267,10000032,21800506,213014.531,2180-05-06 21:30:14+00:00,2180-05-06,PA,None,None,"[35, 32]","['R', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000032,31333364,2180-05-06 19:22:00+00:00,2180-05-06 22:24:00+00:00,F,"OTH SEQUELA, CHR LIV DIS",572.8,OTHER ASCITES,789.59,UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...,070.70,ASYMPTOMATIC HIV INFECTION,V08,None,None,None,None,None,None,None,None,None,None,1,2180-05-06,2180-05-06,F,50 - 60,2013,2180,2180-09-09,0,0,0,0,0,0,0
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,files/p10/p10000032/s50414267/174413ec-4ec4c1f...,50414267,10000032,21800506,213014.531,2180-05-06 21:30:14+00:00,2180-05-06,LATERAL,None,None,"[37, 30]","['A', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000032,31333364,2180-05-06 19:22:00+00:00,2180-05-06 22:24:00+00:00,F,"OTH SEQUELA, CHR LIV DIS",572.8,OTHER ASCITES,789.59,UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...,070.70,ASYMPTOMATIC HIV INFECTION,V08,None,None,None,None,None,None,None,None,None,None,1,2180-05-06,2180-05-06,F,50 - 60,2013,2180,2180-09-09,0,0,0,0,0,0,0
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,53189527,10000032,21800626,165500.312,2180-06-26 16:55:00+00:00,2180-06-26,PA,None,None,"[42, 35]","['R', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000032,36023266,2180-06-26 16:22:00+00:00,2180-06-26 18:28:00+00:00,F,OTHER ASCITES,789.59,UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...,070.70,CIRRHOSIS OF LIVER NOS,571.5,ASYMPTOMATIC HIV INFECTION,V08,None,None,None,None,None,None,None,None,None,None,2,2180-06-26,2180-06-26,F,50 - 60,2013,2180,2180-09-09,0,0,0,0,0,0,0
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,files/p10/p10000032/s53189527/e084de3b-be89b11...,53189527,10000032,21800626,165500.312,2180-06-26 16:55:00+00:00,2180-06-26,LATERAL,None,None,"[42, 34]","['A', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000032,36023266,2180-06-26 16:22:00+00:00,2180-06-26 18:28:00+00:00,F,OTHER ASCITES,789.59,UNSPECIFIED VIRAL HEPATITIS C WITHOUT HEPATIC ...,070.70,CIRRHOSIS OF LIVER NOS,571.5,ASYMPTOMATIC HIV INFECTION,V08,None,None,None,None,None,None,None,None,None,None,2,2180-06-26,2180-06-26,F,50 - 60,2013,2180,2180-09-09,0,0,0,0,0,0,0
4,3be619d1-506a66cf-ff1ab8a1-2efb77bb-fe7d59fc,files/p10/p10000935/s51178377/3be619d1-506a66c...,51178377,10000935,21870823,191426.062,2187-08-23 19:14:26+00:00,2187-08-23,LATERAL,None,None,"[38, 28]","['A', 'F']",DX,CHEST,2048,4096,0.0,1.0,US,NO,10000935,31989916,2187-08-23 16:50:00+00:00,2187-08-23 21:23:00+00:00,F,VOMITING,787.03,OTHER MALAISE AND FATIGUE,780.79,EDEMA,782.3,None,None,None,None,None,None,None,None,None,None,None,None,3,2187-08-23,2187-08-23,F,40 - 50,1997,2171,NaT,0,0,0,0,0,0,0


# Exporting "data"

In [ ]:
# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = 'eye_gaze.cxr2ed_admit'

# #Uncomment to export
# pandas_gbq.to_gbq(data, table_id, project_id=project_id)

1it [00:09,  9.73s/it]


In [ ]:
# Summarizes ED patients who had a CXR exam
sql = """
SELECT 
count(distinct subject_id) as patients
,count(distinct stay_id) as ed_stays
,count(distinct study_id) as cxr_studies
FROM `"""+project_id+""".eye_gaze.cxr2ed_admit` as ed
--WHERE pneumonia = 1
--WHERE CHF = 1
--WHERE Normal = 1
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id, dialect = 'standard')
df

,patients,ed_stays,cxr_studies
0,43408,70273,70457
